In [1]:
import dask.dataframe as ddf
import collections
import pandas as pd
import json
import os
import numpy as np
import re

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
df = pd.read_parquet('../../result/TCM/editor-profile-before-change-format')

In [3]:
df = df.reset_index()

In [4]:
def page_type(title):
    if title.startswith('Talk:'):
        return 'Talk'
    return 'Article'

In [5]:
df['page.type'] = df['page.title'].apply(page_type)

In [6]:
article = df[df['page.type'] == 'Article']

In [7]:
article.columns = article.columns + '.Article'

In [8]:
article = article.rename(columns={'contributor.username.Article':'contributor.username'})

In [9]:
article = article.groupby('contributor.username').sum()

In [10]:
# talk

In [11]:
talk = df[df['page.type'] == 'Talk']

In [12]:
talk.columns = talk.columns + '.Talk'

In [13]:
talk = talk.rename(columns={'contributor.username.Talk':'contributor.username'})

In [14]:
talk = talk.groupby('contributor.username').sum()

In [15]:
article.index.name = ''
talk.index.name = ''

In [16]:
article = article.T
talk = talk.T

In [17]:
res = pd.concat([article,talk]).fillna(0)

In [18]:
res = res.drop(["rules.Article","ban.Article","arbcom.Article"])

In [19]:
res.to_csv("../../result/TCM/TCM-editor-profile-merge-pages.tsv", sep="\t",encoding='utf-16')
res = ddf.from_pandas(res,chunksize=1000)
res.to_parquet('../../result/TCM/TCM-editor-profile-merge-pages')